In [1]:
# -*- coding: utf-8 -*-
"""
IVDD binary training (ivdd vs normal) - train2 (tail_set-centered + per-file min-max)
train1 からの反映点:
- TXTログ保存: 設定パラメータ / 学習フレーム数 / 各epochの train/val loss & acc / classification report
- ラベル判定: one/two を ivdd として扱う（normal は normal）
既存の train2 特徴:
- プロジェクトルート自動検出（scripts配下にdataが出来ない）
- 学習曲線: loss/acc を1枚のPNGで保存 (fig/curve_YYYYMMDD-HHMMSS.png)
- 学習モデル: train2_model に YYYYMMDD-HHMMSS ベース名で best/final を保存
- Val誤分類ウィンドウ一覧を train/val_misclassified/val_misclassified_YYYYMMDD-HHMMSS.csv で保存
- 正規化（tail_set原点 + 各次元min-max）はそのまま維持
"""

# ===== 安定運用: GPU無効/ログ控えめ/スレッド抑制（必要なら） =====
import os
os.environ.setdefault("CUDA_VISIBLE_DEVICES", "-1")
os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "2")
os.environ.setdefault("OMP_NUM_THREADS", "1")
os.environ.setdefault("MKL_NUM_THREADS", "1")

import re, glob
from pathlib import Path
from datetime import datetime
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

print("TF:", tf.__version__)
tf.random.set_seed(42)
np.random.seed(42)

# ========= ルート自動検出（scripts直下で実行してもOK） =========
def detect_project_root() -> Path:
    env = os.environ.get("IVDD_PROJECT_ROOT")
    if env:
        p = Path(env).expanduser().resolve()
        if (p / "data").is_dir() or (p / "scripts").is_dir():
            return p
    try:
        here = Path(__file__).resolve()
        if here.parent.name == "scripts":
            cand = here.parent.parent
            if (cand / "data").is_dir() or (cand / "scripts").is_dir():
                return cand
        if (here.parent / "data").is_dir() or (here.parent / "scripts").is_dir():
            return here.parent
    except NameError:
        pass
    cwd = Path.cwd()
    for cand in [cwd] + list(cwd.parents):
        if (cand / "data").is_dir() and (cand / "scripts").is_dir():
            return cand
    return cwd.parent if cwd.name == "scripts" else cwd

PROJ_ROOT     = detect_project_root()
TRAIN_DIR     = PROJ_ROOT / "data" / "train"
TRAIN_CSV_DIR = TRAIN_DIR / "train_csv"
FIG_DIR       = TRAIN_DIR / "fig"
MODEL_DIR     = TRAIN_DIR / "train2_model"
VALERR_DIR    = TRAIN_DIR / "val_misclassified"
LOG_DIR       = TRAIN_DIR / "train_logs"          # ★ 新設: テキストログ置き場
for d in [FIG_DIR, MODEL_DIR, VALERR_DIR, LOG_DIR]:
    d.mkdir(parents=True, exist_ok=True)

CSV_GLOB = str(TRAIN_CSV_DIR / "*.csv")

# ========= パラメータ =========
KEYPOINTS = [
    "tail_set",
    "right_tarsal",
    "right_paw",
    "left_tarsal",
    "left_paw",
]
USE_LIKELIHOOD      = False
MIN_KEEP_LIKELIHOOD = 0.6

SEQ_LEN   = 60
STRIDE    = 30
DIMS      = len(KEYPOINTS) * 2   # =10
N_HIDDEN  = 30

BATCH_SIZE = 30
EPOCHS     = 50
LR         = 1e-4
L2_LAMBDA  = 1e-4

VAL_SPLIT_BY_FILE = True

# バイナリ: 0=normal, 1=ivdd
CLASS_NAMES  = ["normal", "ivdd"]
CLASS_TO_IDX = {"normal": 0, "ivdd": 1}

# ランごとユニークID（日時）
DATE_STR = datetime.now().strftime("%Y%m%d-%H%M%S")

# ========= ラベル推定 =========
def infer_label_from_filename(path: str) -> int:
    """
    ファイル名や親ディレクトリからラベルを推定。
    - one / two は ivdd として扱う
    - ivdd*, normal* などの先頭トークンも許可
    - それ以外でも 'ivdd' / 'normal' の厳密一致トークンがあれば採用
    """
    base = os.path.basename(path).lower()
    stem = os.path.splitext(base)[0]
    tokens = [t for t in re.split(r'[^a-z0-9]+', stem) if t]

    if tokens:
        head = tokens[0]
        if re.fullmatch(r'(ivdd|one|two)\d*', head):   # 例: ivdd1, one2 もOK
            return CLASS_TO_IDX["ivdd"]
        if re.fullmatch(r'normal\d*', head):
            return CLASS_TO_IDX["normal"]

    # 他トークンに厳密一致（片方のみヒット時）
    if (("ivdd" in tokens) or ("one" in tokens) or ("two" in tokens)) and ("normal" not in tokens):
        return CLASS_TO_IDX["ivdd"]
    if ("normal" in tokens) and not (("ivdd" in tokens) or ("one" in tokens) or ("two" in tokens)):
        return CLASS_TO_IDX["normal"]

    # 親ディレクトリも確認
    parent_tokens = [t for t in re.split(r'[^a-z0-9]+', os.path.dirname(path).lower()) if t]
    if (("ivdd" in parent_tokens) or ("one" in parent_tokens) or ("two" in parent_tokens)) and ("normal" not in parent_tokens):
        return CLASS_TO_IDX["ivdd"]
    if ("normal" in parent_tokens) and not (("ivdd" in parent_tokens) or ("one" in parent_tokens) or ("two" in parent_tokens)):
        return CLASS_TO_IDX["normal"]

    # 最後の手段: ivdd/normal の直後が数字なら採用
    if re.search(r'(?<![a-z])ivdd(?=\d)', stem) or re.search(r'(?<![a-z])one(?=\d)', stem) or re.search(r'(?<![a-z])two(?=\d)', stem):
        return CLASS_TO_IDX["ivdd"]
    if re.search(r'(?<![a-z])normal(?=\d)', stem):
        return CLASS_TO_IDX["normal"]

    raise ValueError(f"ラベル不明: {base}（先頭を ivdd_/one_/two_/normal_ などにするか、上の規則に合わせてください）")

# ========= DLC 読み取り & 正規化 =========
def _norm_key(s: str) -> str:
    """比較用: 小文字化 + 空白/アンダー/ハイフンを除去"""
    return "".join(ch for ch in s.lower() if ch not in " _-")

def _resolve_keypoints(all_bodyparts, requested):
    norm2orig = {}
    for bp in all_bodyparts:
        k = _norm_key(bp)
        if k not in norm2orig:
            norm2orig[k] = bp
    resolved, missing = [], []
    for req in requested:
        k = _norm_key(req)
        if k in norm2orig:
            resolved.append(norm2orig[k])
        else:
            missing.append(req)
    if missing:
        raise ValueError(f"指定キーポイントがCSVに見つかりません: {missing}\n利用可能: {all_bodyparts}")
    return resolved

def read_dlc_5kp_xy(csv_path: str, keypoints, use_likelihood=True, min_keep_likelihood=0.6):
    df = pd.read_csv(csv_path, header=[0,1,2], index_col=0)
    bodyparts = list({bp for (_, bp, _) in df.columns})
    use_kps = _resolve_keypoints(bodyparts, keypoints)

    cols = {}
    for bp in use_kps:
        cols[f"{bp}_x"] = df.xs((bp, "x"), level=[1,2], axis=1)
        cols[f"{bp}_y"] = df.xs((bp, "y"), level=[1,2], axis=1)
    X_df = pd.concat(cols.values(), axis=1)
    X_df.columns = list(cols.keys())

    if use_likelihood:
        for bp in use_kps:
            try:
                lcol = df.xs((bp, "likelihood"), level=[1,2], axis=1).values.flatten()
                low = lcol < min_keep_likelihood
                for c in [f"{bp}_x", f"{bp}_y"]:
                    v = X_df[c].values
                    v[low] = np.nan
                    X_df[c] = v
            except KeyError:
                pass

    X_df = X_df.interpolate(method="linear", limit_direction="both", axis=0)
    X_df = X_df.bfill().ffill().fillna(0.0)
    X = X_df.values.astype(np.float32)  # (T, DIMS)
    return X, use_kps

def normalize_tailset_minmax(X: np.ndarray, used_kps, ref_name: str = "tail_set", eps: float = 1e-6) -> np.ndarray:
    """
    1) tail_set を原点に平行移動（各フレーム）
    2) その後、各次元で min-max 正規化（ファイル単位）
    ※ マッチは _norm_key で行うので 'tail set' / 'tail_set' どちらでもOK
    """
    norm_used = [_norm_key(s) for s in used_kps]
    ref_key = _norm_key(ref_name)
    if ref_key not in norm_used:
        raise ValueError(f"'{ref_name}' が used_kps に見つかりません: {used_kps}")
    ref_idx = norm_used.index(ref_key)

    # 原点平行移動
    cx = X[:, 2*ref_idx]
    cy = X[:, 2*ref_idx + 1]
    Xc = X.copy()
    for i in range(len(used_kps)):
        Xc[:, 2*i]   -= cx
        Xc[:, 2*i+1] -= cy

    # min-max
    x_min = Xc.min(axis=0, keepdims=True)
    x_max = Xc.max(axis=0, keepdims=True)
    Xn = (Xc - x_min) / (x_max - x_min + eps)
    return Xn

def make_windows(X: np.ndarray, seq_len: int, stride: int):
    n = X.shape[0]
    if n < seq_len:
        return np.empty((0, seq_len, X.shape[1]), dtype=X.dtype), []
    starts = list(range(0, n - seq_len + 1, stride))
    Xw = np.stack([X[s:s+seq_len] for s in starts], axis=0) if starts \
         else np.empty((0, seq_len, X.shape[1]), dtype=X.dtype)
    return Xw, starts

def build_dataset(csv_paths, seq_len=SEQ_LEN, stride=STRIDE):
    """
    戻り:
      X: (N, T, D)
      y: (N,) 0/1
      file_ids: (N,) 元CSVファイル名
      starts: (N,) ウィンドウ開始フレーム
    """
    X_list, y_list, file_ids, starts_all = [], [], [], []
    used_kps_any = None

    for p in csv_paths:
        y_lab = infer_label_from_filename(p)
        X_raw, used_kps = read_dlc_5kp_xy(p, keypoints=KEYPOINTS,
                                          use_likelihood=USE_LIKELIHOOD,
                                          min_keep_likelihood=MIN_KEEP_LIKELIHOOD)
        if used_kps_any is None:
            used_kps_any = used_kps
        if X_raw.shape[1] != DIMS:
            raise ValueError(f"{Path(p).name}: 次元 {X_raw.shape[1]} != 期待 {DIMS}")

        # ★ tail_set中心 + min-max 正規化
        X_norm = normalize_tailset_minmax(X_raw, used_kps, ref_name="tail_set")

        X_win, starts = make_windows(X_norm, seq_len, stride)
        if X_win.shape[0] == 0:
            print(f"[WARN] {Path(p).name}: フレーム不足（{seq_len}未満）でスキップ")
            continue

        X_list.append(X_win)
        y_list.append(np.full((X_win.shape[0],), y_lab, dtype=np.int64))
        file_ids += [Path(p).name] * X_win.shape[0]
        starts_all += starts

    if not X_list:
        raise RuntimeError("データが作れませんでした。CSV名に normal/ivdd/one/two を含めてください。")

    X = np.concatenate(X_list, axis=0)
    y = np.concatenate(y_list, axis=0)
    file_ids  = np.array(file_ids)
    starts_all = np.array(starts_all, dtype=int)
    print(f"[INFO] 使用キーポイント実名: {used_kps_any}")
    return X, y, file_ids, starts_all

# ========= モデル =========
def build_lstm_model(seq_len: int, dims: int, n_hidden: int, l2_lambda: float = 1e-4) -> keras.Model:
    reg = keras.regularizers.l2(l2_lambda)
    inp = keras.Input(shape=(seq_len, dims), name="input")
    td  = keras.layers.TimeDistributed(
        keras.layers.Dense(n_hidden, activation="relu", kernel_regularizer=reg),
        name="td_dense"
    )(inp)
    x   = keras.layers.LSTM(n_hidden, return_sequences=True, kernel_regularizer=reg, name="lstm1")(td)
    x   = keras.layers.LSTM(n_hidden, kernel_regularizer=reg, name="lstm2")(x)
    out = keras.layers.Dense(1, kernel_regularizer=reg, name="logits")(x)  # from_logits=True
    return keras.Model(inp, out, name="ivdd_lstm")

# ========= データ読み込み =========
csv_files = sorted(glob.glob(CSV_GLOB))
if not csv_files:
    raise FileNotFoundError(f"CSV が見つかりません: {CSV_GLOB}")

X, y, file_ids, starts = build_dataset(csv_files, SEQ_LEN, STRIDE)
print("X:", X.shape, "y:", y.shape, "files:", len(np.unique(file_ids)))

# ========= 分割（ファイル単位推奨） =========
if VAL_SPLIT_BY_FILE:
    uniq = np.unique(file_ids)
    tr_files, va_files = train_test_split(uniq, test_size=0.2, random_state=42, shuffle=True)
    tr_mask = np.isin(file_ids, tr_files)
    va_mask = np.isin(file_ids, va_files)
    X_train, y_train = X[tr_mask], y[tr_mask]
    X_val,   y_val   = X[va_mask], y[va_mask]
    val_file_ids     = file_ids[va_mask]
    val_starts       = starts[va_mask]
else:
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    val_file_ids = np.array(["unknown.csv"]*len(y_val))
    val_starts   = np.arange(len(y_val))

print("train:", X_train.shape, "val:", X_val.shape)

# ========= クラス不均衡対策 =========
unique_classes = np.unique(y_train)
if set(unique_classes) == {0, 1}:
    cls_w = compute_class_weight(class_weight="balanced", classes=np.array([0,1]), y=y_train)
    class_weight = {0: float(cls_w[0]), 1: float(cls_w[1])}
else:
    class_weight = None
print("class_weight:", class_weight)

# ========= 学習（EarlyStoppingなし） =========
model = build_lstm_model(SEQ_LEN, DIMS, N_HIDDEN, L2_LAMBDA)
model.summary()

opt = keras.optimizers.Adam(learning_rate=LR)
loss_fn = keras.losses.BinaryCrossentropy(from_logits=True)
metrics = [keras.metrics.BinaryAccuracy(name="accuracy")]
model.compile(optimizer=opt, loss=loss_fn, metrics=metrics)

best_model_path  = MODEL_DIR / f"ivdd_lstm_{DATE_STR}_best.keras"
final_model_path = MODEL_DIR / f"ivdd_lstm_{DATE_STR}_final.keras"

callbacks = [
    keras.callbacks.ModelCheckpoint(
        str(best_model_path), monitor="val_accuracy",
        save_best_only=True, save_weights_only=False, mode="max", verbose=1
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", mode="min",
        factor=0.5, patience=5, min_lr=1e-5, verbose=1
    ),
]

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    class_weight=class_weight,
    verbose=1,
    callbacks=callbacks
)

# ========= 学習曲線（loss/acc を1枚で保存） =========
curve_png = FIG_DIR / f"curve_{DATE_STR}.png"
plt.figure(figsize=(10,4))
plt.subplot(1,2,1); plt.plot(history.history["loss"], label="train"); plt.plot(history.history["val_loss"], label="val")
plt.title("Loss"); plt.legend()
plt.subplot(1,2,2); plt.plot(history.history["accuracy"], label="train"); plt.plot(history.history["val_accuracy"], label="val")
plt.title("Accuracy"); plt.legend()
plt.tight_layout(); plt.savefig(curve_png, dpi=150); plt.close()
print(f"[INFO] saved curve: {curve_png}")

# ========= Val評価 & 誤分類CSV =========
logits_val = model.predict(X_val, batch_size=64)
p_ivdd = tf.math.sigmoid(logits_val).numpy().ravel()   # ivdd=1 の確率
y_pred = (p_ivdd >= 0.5).astype(int)

print("\n[Window-level] classification_report:")
cls_report_str = classification_report(y_val, y_pred, target_names=CLASS_NAMES, digits=4)
print(cls_report_str)
cm = confusion_matrix(y_val, y_pred)
print("[Window-level] confusion matrix:\n", cm)

# 誤分類ウィンドウを書き出し
val_df = pd.DataFrame({
    "file": val_file_ids,
    "win_start": val_starts,           # ウィンドウ開始フレーム
    "true": y_val,
    "pred": y_pred,
    "p_ivdd": p_ivdd,
    "p_normal": 1.0 - p_ivdd,
})
errors_df = val_df[val_df["true"] != val_df["pred"]].copy()
err_csv = VALERR_DIR / f"val_misclassified_{DATE_STR}.csv"
errors_df.to_csv(err_csv, index=False, encoding="utf-8-sig")
print(f"[INFO] saved misclassified windows: {err_csv}  (rows={len(errors_df)})")

# ========= 最終モデル保存 =========
model.save(str(final_model_path))
print(f"[INFO] saved final model to: {final_model_path}")

# ========= TXTログ保存（train1の拡張を反映） =========
log_txt = LOG_DIR / f"train2_log_{DATE_STR}.txt"
try:
    with open(log_txt, "w", encoding="utf-8") as f:
        # 1) 設定パラメータ
        f.write("[PARAMS]\n")
        f.write(f"DATE_STR     : {DATE_STR}\n")
        f.write(f"PROJ_ROOT    : {PROJ_ROOT}\n")
        f.write(f"TRAIN_CSV_DIR: {TRAIN_CSV_DIR}\n")
        f.write(f"KEYPOINTS    : {KEYPOINTS}\n")
        f.write(f"USE_LIKELIHOOD/MIN_KEEP : {USE_LIKELIHOOD}/{MIN_KEEP_LIKELIHOOD}\n")
        f.write(f"SEQ_LEN/STRIDE: {SEQ_LEN}/{STRIDE}\n")
        f.write(f"N_HIDDEN/L2   : {N_HIDDEN}/{L2_LAMBDA}\n")
        f.write(f"BATCH_SIZE/EPOCHS/LR: {BATCH_SIZE}/{EPOCHS}/{LR}\n")
        f.write(f"VAL_SPLIT_BY_FILE: {VAL_SPLIT_BY_FILE}\n")
        f.write(f"CLASS_NAMES      : {CLASS_NAMES}\n")
        f.write(f"class_weight     : {class_weight}\n")
        f.write("\n")

        # 2) データ統計 & 学習フレーム数
        n_train_win = int(X_train.shape[0])
        n_val_win   = int(X_val.shape[0])
        approx_train_frames = n_train_win * SEQ_LEN
        approx_val_frames   = n_val_win * SEQ_LEN
        f.write("[DATA]\n")
        f.write(f"#CSV files        : {len(csv_files)}\n")
        f.write(f"#train windows    : {n_train_win}\n")
        f.write(f"#val windows      : {n_val_win}\n")
        f.write(f"approx train frames (windows*SEQ_LEN): {approx_train_frames}\n")
        f.write(f"approx val frames   (windows*SEQ_LEN): {approx_val_frames}\n")
        f.write("\n")

        # 3) 各エポックのメトリクス
        f.write("[EPOCH METRICS]\n")
        keys = list(history.history.keys())  # ['loss','accuracy','val_loss','val_accuracy',...]
        f.write(f"keys: {keys}\n")
        f.write("epoch,loss,accuracy,val_loss,val_accuracy\n")
        for i in range(EPOCHS):
            loss = history.history.get("loss", [None]*EPOCHS)[i]
            acc  = history.history.get("accuracy", [None]*EPOCHS)[i]
            vls  = history.history.get("val_loss", [None]*EPOCHS)[i]
            vacc = history.history.get("val_accuracy", [None]*EPOCHS)[i]
            f.write(f"{i+1},{loss},{acc},{vls},{vacc}\n")
        f.write("\n")

        # 4) 最終 classification report
        f.write("[WINDOW-LEVEL CLASSIFICATION REPORT]\n")
        f.write(cls_report_str + "\n\n")

        # 5) 混同行列
        f.write("[WINDOW-LEVEL CONFUSION MATRIX]\n")
        f.write(np.array2string(cm, threshold=10000) + "\n")

        # 6) 生成物のパス
        f.write("\n[ARTIFACTS]\n")
        f.write(f"curve_png       : {curve_png}\n")
        f.write(f"best_model_path : {best_model_path}\n")
        f.write(f"final_model_path: {final_model_path}\n")
        f.write(f"val_errors_csv  : {err_csv}\n")
    print(f"[INFO] saved TXT log: {log_txt}")
except Exception as e:
    print(f"[WARN] TXTログの保存に失敗しました: {e}")

print("[DONE] training complete.")


TF: 2.19.0
[INFO] 使用キーポイント実名: ['tail_set', 'right_tarsal', 'right_paw', 'left_tarsal', 'left_paw']
X: (420, 60, 10) y: (420,) files: 82
train: (349, 60, 10) val: (71, 60, 10)
class_weight: {0: 0.9914772727272727, 1: 1.0086705202312138}


Model: "ivdd_lstm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input (InputLayer)              │ (None, 60, 10)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ td_dense (TimeDistributed)      │ (None, 60, 30)         │           330 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm1 (LSTM)                    │ (None, 60, 30)         │         7,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm2 (LSTM)                    │ (None, 30)             │         7,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ logits (Dense)                  │ (None, 1)              │            31 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,001 (58.60 KB)

 Trainable params: 15,001 (58.60 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
 6/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4322 - loss: 0.7058
Epoch 1: val_accuracy improved from -inf to 0.67606, saving model to c:\kanno\vscode\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\data\train\train2_model\ivdd_lstm_20251225-162504_best.keras
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 52ms/step - accuracy: 0.4601 - loss: 0.7055 - val_accuracy: 0.6761 - val_loss: 0.7131 - learning_rate: 1.0000e-04
Epoch 2/50
 7/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4371 - loss: 0.7020
Epoch 2: val_accuracy did not improve from 0.67606
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.4601 - loss: 0.7026 - val_accuracy: 0.6761 - val_loss: 0.7097 - learning_rate: 1.0000e-04
Epoch 3/50
 7/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4371 - loss: 0.7019
Epoch 3: val_accuracy did not improve from 0.67606
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.4601 - loss: 0.7021 - val_accur